# Reproducing Larichev & Held JPO 1995
 Eddy Amplitudes and Fluxes in a Homogeneous Model of Fully Developed Baroclinic Instability. J. Phys. Oceanogr., 25, 2285–2297. doi: [http://dx.doi.org/10.1175](http://journals.ametsoc.org/doi/pdf/10.1175/1520-0485%281995%29025%3C2285%3AEAAFIA%3E2.0.CO%3B2).

In [3]:
import numpy as np
from numpy import pi
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline

import pyqg
from pyqg import diagnostic_tools

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 21 days
Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 21 days
Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 21 days


In [4]:
Nx = 256          # number of grid points
Nz = 2            # number of layer
L =  2*pi         # length scale of box    
Ld = 3./50.       # deformation scale      
kd = 1./Ld        # deformation wavenumber 
H1 = 0.5          # upper layer thickness  
H2 = 0.5          # lower layer thickness 
delta = H1/H2

U1 = .005          # upper layer zonal velocity 
U2 = -.005         # lower layer zonal velicity 
Us = U1-U2         # vertical shear             
rek = .04          # linear bottom drag coeff.  

Ti = Ld/(abs(Us))  # estimate of most unstable e-folding time scale 

dt = Ti/500.      # time-step 

In [5]:
def init_condition(model,sig=1.e-7):
    """ White noise spectrum with amplitude sig """
    return sig*np.random.randn(model.nz,model.nx,model.ny)

In [6]:
def psi_tau_h(ph):
    """ calculates barotropic and
            baroclinic streamfunction """
    return (ph[0]+ph[1])/2., (ph[0]-ph[1])/2.

def calc_tfh(pph,pth,U,k):
    """ calculates thickness flux,
        or production term """
    return  -U*(m.rd**-2)*(1j*k*pph.conj()*pth).real

In [7]:
m = pyqg.LayeredModel(nx=Nx, nz=Nz, U = [U1,U2], V=[0.,0.],
                              L=L,f=1.,beta=0., rd=Ld, H = [H1,H2],rek=rek,
                              dt=0.001,tmax=500.,twrite=5000, 
                              tavestart=800.,taveint=1., ntd=4, delta=delta,
                              )

m.set_q(init_condition(m, sig=1.e-1))

INFO:  Logger initialized
INFO:  Kernel initialized


In [ ]:
ke = []
t  = []

for i in m.run_with_snapshots(tsnapstart=0, tsnapint=1.):

    ke.append(m._calc_ke())
    t.append(m.t)

    pph, pth = psi_tau_h(m.ph)

    # calculates the production term
    prod = calc_tfh(pph,pth,m.Ubg[0],m.k)

    ki, prod = diagnostic_tools.calc_ispec(m,prod)

    try:
        prod_k = np.vstack([prod_k,prod])
    except:
        prod_k = prod
    

INFO: Step: 5000, Time: 5.00e+00, KE: 1.16e-06, CFL: 0.000


# Snapshots

In [ ]:
qph, qth = psi_tau_h(m.qh)

plt.figure(figsize=(14,5))

plt.subplot(121)
plt.pcolormesh(m.x,m.y,np.fft.irfft2(qph))
plt.colorbar()
plt.clim(-25,25)
plt.xlim(0,2*pi)
plt.ylim(0,2*pi)
plt.title('Barotropic PV')

plt.subplot(122)
plt.pcolormesh(m.x,m.y,np.fft.irfft2(qth)+(m.Qy[0]-m.Qy[1])*m.y)
plt.colorbar()
plt.clim(0,180)
plt.xlim(0,2*pi)
plt.ylim(0,2*pi)
plt.title('Baroclinic PV')

In [1]:
ki, ti = np.meshgrid(ki,t)

NameError: name 'np' is not defined

#  The spectral structure of the production as a function of time during the spinup stage: LH95 figure 1.

In [2]:
plt.figure(figsize=(13,9))
plt.pcolormesh(ki,ti,prod_k/m.M**2, norm = LogNorm())
plt.xscale('log')
plt.xlim(1.,1e2)
plt.ylim(1,300)
plt.clim(1.e-7,1.e-3)
plt.xlabel('Wavenumber',fontsize=20.)
plt.xticks([1.,10.,50.,100.])
plt.ylabel('time',fontsize=20.)
plt.colorbar(label=r'Spectral density of APE production rate')

NameError: name 'plt' is not defined